<a href="https://colab.research.google.com/github/cbonnin88/TheLook/blob/main/Thelook_ecommerce_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import polars as pl
import plotly.express as px
import os

In [38]:
# Loading the Data
df_thelook = pl.read_csv('thelook_funnel_data.csv')

In [39]:
df_thelook.head()

user_id,session_id,event_timestamp,event_type,traffic_source,browser,country,gender,product_category,retail_price
i64,str,str,str,str,str,str,str,str,f64
5,"""55614f8b-05c4-40db-bf89-982ea3…","""2025-11-24 04:34:12 UTC""","""product""","""Adwords""","""Chrome""","""China""","""M""","""Outerwear & Coats""",199.660004
5,"""55614f8b-05c4-40db-bf89-982ea3…","""2025-11-24 04:34:12 UTC""","""product""","""Adwords""","""Chrome""","""China""","""M""","""Outerwear & Coats""",199.660004
5,"""55614f8b-05c4-40db-bf89-982ea3…","""2025-11-24 04:34:12 UTC""","""product""","""Adwords""","""Chrome""","""China""","""M""","""Outerwear & Coats""",199.660004
5,"""55614f8b-05c4-40db-bf89-982ea3…","""2025-11-24 04:34:12 UTC""","""product""","""Adwords""","""Chrome""","""China""","""M""","""Outerwear & Coats""",199.660004
5,"""55614f8b-05c4-40db-bf89-982ea3…","""2025-11-24 04:34:12 UTC""","""product""","""Adwords""","""Chrome""","""China""","""M""","""Outerwear & Coats""",199.660004


In [40]:
print(f'Number of Rows: {df_thelook.shape[0]}')
print(f'Number of Columns: {df_thelook.shape[1]}')

Number of Rows: 1278248
Number of Columns: 10


# **Cleaning the Data**

In [41]:
df_thelook = df_thelook.with_columns(
    pl.col('event_timestamp').str.to_datetime(format="%Y-%m-%d %H:%M:%S%.f UTC", strict=False)
)

print(f'Data Loaded: {df_thelook.height} rows')
print(f'Data Range: {df_thelook['event_timestamp'].min()} to {df_thelook['event_timestamp'].max()}')

Data Loaded: 1278248 rows
Data Range: 2025-10-11 10:16:36 to 2026-01-13 00:19:42.096562


# **Feature Engineering: Session & Retention Metrics**

In [42]:
# A. Session Flow: Time since previous event
df_thelook_sessions = df_thelook.sort(['user_id','event_timestamp']).with_columns(
    (pl.col('event_timestamp') - pl.col('event_timestamp').shift(1))
    .over('session_id')
    .alias('time_since_last_event')
)

display(df_thelook_sessions.head())

user_id,session_id,event_timestamp,event_type,traffic_source,browser,country,gender,product_category,retail_price,time_since_last_event
i64,str,datetime[μs],str,str,str,str,str,str,f64,duration[μs]
5,"""55614f8b-05c4-40db-bf89-982ea3…",2025-11-24 04:34:12,"""product""","""Adwords""","""Chrome""","""China""","""M""","""Outerwear & Coats""",199.660004,null
5,"""55614f8b-05c4-40db-bf89-982ea3…",2025-11-24 04:34:12,"""product""","""Adwords""","""Chrome""","""China""","""M""","""Outerwear & Coats""",199.660004,0µs
5,"""55614f8b-05c4-40db-bf89-982ea3…",2025-11-24 04:34:12,"""product""","""Adwords""","""Chrome""","""China""","""M""","""Outerwear & Coats""",199.660004,0µs
5,"""55614f8b-05c4-40db-bf89-982ea3…",2025-11-24 04:34:12,"""product""","""Adwords""","""Chrome""","""China""","""M""","""Outerwear & Coats""",199.660004,0µs
5,"""55614f8b-05c4-40db-bf89-982ea3…",2025-11-24 04:34:12,"""product""","""Adwords""","""Chrome""","""China""","""M""","""Outerwear & Coats""",199.660004,0µs


In [43]:
# B. Retention: Days Between Orders
purchase_df = (
    df_thelook.filter(pl.col('event_type') == 'purchase')
    .sort(['user_id','event_timestamp'])
    .with_columns([
        (pl.col('event_timestamp') - pl.col('event_timestamp').shift(1))
        .over('user_id')
        .dt.total_days()
        .alias('days_between_orders')
    ])
    .filter(pl.col('days_between_orders').is_not_null())
)

display(purchase_df.head())

user_id,session_id,event_timestamp,event_type,traffic_source,browser,country,gender,product_category,retail_price,days_between_orders
i64,str,datetime[μs],str,str,str,str,str,str,f64,i64
8,"""7e3d45cb-048c-4fe1-9698-c4e4be…",2025-10-29 08:56:18,"""purchase""","""Adwords""","""Firefox""","""Spain""","""M""",null,null,1
8,"""261f8d1f-ad24-4810-ae2c-241387…",2025-10-30 09:29:30,"""purchase""","""Adwords""","""Chrome""","""Spain""","""M""",null,null,1
25,"""5bbd4835-97a6-417c-a22c-d8a361…",2025-12-14 20:55:57,"""purchase""","""Adwords""","""Chrome""","""China""","""M""",null,null,1
26,"""6b4052df-b7f6-4191-8754-979692…",2025-10-24 00:35:52,"""purchase""","""Email""","""Firefox""","""United States""","""M""",null,null,0
44,"""965afdf7-8b73-4dbc-8fdb-3d2efa…",2025-12-03 10:36:29,"""purchase""","""Email""","""Chrome""","""Brasil""","""F""",null,null,25


# **Visualizations**

**Days Between Repeat Purchases (Retention)**

In [44]:
median_days = purchase_df['days_between_orders'].median()
display(median_days)

1.0

In [45]:
fig_retention = px.histogram(
    purchase_df.to_pandas(),
    x='days_between_orders',
    nbins=30,
    title=f'Customer Retention Cycle (Median: {median_days:.1f} Days)',
    labels = {'day_between_orders': 'Days Since Last Order'},
    template='plotly_white',
    color_discrete_sequence=["#636EFA"]
)

fig_retention.add_vline(x=median_days, line_dash='dash',line_color='red')
fig_retention.show()

**Peak Purchasing Times**

In [46]:
purchasing_time_df = (
    df_thelook.filter(pl.col('event_type') == 'purchase')
    .with_columns([
        pl.col('event_timestamp').dt.strftime('%A').alias('day_name'),
        pl.col('event_timestamp').dt.hour().alias('hour_of_day')
    ])
)

display(purchasing_time_df)

user_id,session_id,event_timestamp,event_type,traffic_source,browser,country,gender,product_category,retail_price,day_name,hour_of_day
i64,str,datetime[μs],str,str,str,str,str,str,f64,str,i8
5,"""55614f8b-05c4-40db-bf89-982ea3…",2025-11-24 04:36:16,"""purchase""","""Adwords""","""Chrome""","""China""","""M""",null,null,"""Monday""",4
8,"""5a25cd8c-209a-4ac8-ad1d-6e0bd4…",2025-10-27 10:09:53,"""purchase""","""Adwords""","""Chrome""","""Spain""","""M""",null,null,"""Monday""",10
8,"""7e3d45cb-048c-4fe1-9698-c4e4be…",2025-10-29 08:56:18,"""purchase""","""Adwords""","""Firefox""","""Spain""","""M""",null,null,"""Wednesday""",8
8,"""261f8d1f-ad24-4810-ae2c-241387…",2025-10-30 09:29:30,"""purchase""","""Adwords""","""Chrome""","""Spain""","""M""",null,null,"""Thursday""",9
21,"""e7526b4a-06e7-4c71-8c72-116fe6…",2025-10-12 09:04:58,"""purchase""","""Email""","""Firefox""","""United Kingdom""","""M""",null,null,"""Sunday""",9
…,…,…,…,…,…,…,…,…,…,…,…
99991,"""0da847fd-26e1-49ac-8243-338d29…",2025-12-29 18:31:38,"""purchase""","""Adwords""","""Safari""","""Brasil""","""F""",null,null,"""Monday""",18
99994,"""1b318cc3-bc06-449d-9ab7-bc882a…",2025-11-01 23:46:14,"""purchase""","""Email""","""Other""","""United States""","""F""",null,null,"""Saturday""",23
99995,"""d821c461-7974-4eba-937e-153512…",2025-11-24 09:53:04,"""purchase""","""Adwords""","""Chrome""","""United Kingdom""","""M""",null,null,"""Monday""",9


In [47]:
# Usings a defined category order so that the days of the week are in calendar order
day_order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

In [48]:
fig_purchase = px.density_heatmap(
    purchasing_time_df.to_pandas(),
    x='hour_of_day',
    y='day_name',
    title='Peak Purchasing Times',
    labels={'hour_of_day':'Hour of Day (24H)','day_name':'Day of Week'},
    category_orders = {'day_name':day_order},
    color_continuous_scale='Viridis',
    nbinsx=24,
    nbinsy=7
)

fig_purchase.update_layout(
    xaxis=dict(dtick=2),
    template='plotly_white'
)

fig_purchase.show()

# **Export for Amplitude**

In [53]:
# Mapping the new sql columns (gender, browser) to Amplitude's schema
amplitude_export = df_thelook.select([
    pl.col('user_id'),
    pl.col('event_type'),
    pl.col('event_timestamp').dt.strftime('%Y-%m-%d %H:%M:%S').alias('time'),
    # User Properties
    pl.col('country'),
    pl.col('gender'),
    pl.col('browser'),
    pl.col('traffic_source'),
    # Event Properties
    pl.col('product_category'),
    pl.col('retail_price')
])

amplitude_export.write_csv('amplitude_import.csv')
print('\nSuccess: amplitude_import.csv created with Gender and Browser data')


Success: amplitude_import.csv created with Gender and Browser data


In [54]:
filename = "amplitude_import.csv"
file_size_mb = os.path.getsize(filename) / (1024 * 1024)

print(f"\nChecking file size: {file_size_mb:.2f} MB")

if file_size_mb > 9.5: # Safety buffer below 10MB
    print("File is too large for single upload. Splitting...")

    # Reload the full file
    full_df = pl.read_csv(filename)
    total_rows = full_df.height

    # Calculate chunk size (approx 50k rows usually fits 10MB, but let's be dynamic)
    # A simple approach is to split into N parts
    num_parts = int(file_size_mb // 9) + 1
    chunk_size = total_rows // num_parts

    print(f"Splitting into {num_parts} files...")

    for i in range(num_parts):
        start = i * chunk_size
        # For the last chunk, grab everything remaining
        if i == num_parts - 1:
            chunk = full_df.slice(start, total_rows - start)
        else:
            chunk = full_df.slice(start, chunk_size)

        output_name = f"amplitude_import_part_{i+1}.csv"
        chunk.write_csv(output_name)
        print(f" -> Created: {output_name}")

    print("\nDone! Upload these part files one by one to Amplitude.")
else:
    print("File is under 10MB. Good to go!")


Checking file size: 98.46 MB
File is too large for single upload. Splitting...
Splitting into 11 files...
 -> Created: amplitude_import_part_1.csv
 -> Created: amplitude_import_part_2.csv
 -> Created: amplitude_import_part_3.csv
 -> Created: amplitude_import_part_4.csv
 -> Created: amplitude_import_part_5.csv
 -> Created: amplitude_import_part_6.csv
 -> Created: amplitude_import_part_7.csv
 -> Created: amplitude_import_part_8.csv
 -> Created: amplitude_import_part_9.csv
 -> Created: amplitude_import_part_10.csv
 -> Created: amplitude_import_part_11.csv

Done! Upload these part files one by one to Amplitude.
